In [1]:
import pandas as pd    
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List
import math
import re
import os
from tqdm.auto import tqdm
%matplotlib inline

In [2]:
path = path = os.path.abspath(os.path.join(os.path.dirname('EDA.ipynb'), '..', 'Parsing\\dataframes'))

Грузим все источники данных.

In [3]:
# Парсинг
mtgtrade_df = pd.read_csv(path +'/mtgtrade_parsing.csv')
scryfall_df = pd.read_csv(path +'/scryfall_parsing.csv')
# Справочники
mtg_sets_df = pd.read_excel(path +'/MTG_sets.xlsx', date_parser = pd.to_datetime)
mtg_abilities = pd.read_excel(path +'/MTG_abilities.xlsx').values.tolist()

In [4]:
scryfall_df.drop_duplicates(inplace = True)
mtgtrade_df.drop_duplicates(inplace = True)

In [5]:
mtgtrade_df['Name'] = mtgtrade_df[['Language', 'Name1', 'Name2']].apply(lambda x: x['Name1'] if x['Language'] == 'Английский' else x['Name2'], axis = 1)

Объединение в 1 таблицу

In [6]:
def create_ability_features(abilities:List[List[str]]) -> pd.DataFrame:
    '''
    Получение признаков о наличии механик
    '''
    features = df['Feature'].values.tolist()
    k = []
    for ability in tqdm(abilities):
        # k = features.apply(lambda x: 1 if type(x) != float and ability in x else 0)
        k.append([1 if (type(x) != float and ability[0].lower() in x) else 0 for x in features])
    colnames = [x[1]+'_'+ x[0] for x in abilities]
    k = map(list, zip(*k))
    feat_df = pd.DataFrame(k, columns = colnames)
    return feat_df

In [7]:
df = mtgtrade_df.merge(scryfall_df, on = 'Name')
df = df.merge(mtg_sets_df,left_on='Set', right_on='Name')
df = pd.concat([df, create_ability_features(mtg_abilities)], axis = 1)

Первичное создание признаков

In [8]:
# Базовые преобразования
df = df.applymap(lambda x: 'Other' if x == '[]' else x)
df['Feature'].fillna('', inplace = True)
df['Feature'] = df['Feature'].str.lower()

In [9]:
# Длина описания как признак
df['Feature length'] = df['Feature'].apply(lambda x: len(x) if type(x) != float else 0)

In [10]:
# Вытаскиваем главный тип карты
card_types = ['basic land', 'artifact creature', 'creature', 'instant', 'sorcery', 'enchantment creature', 'enchantment', 'land', 
             'artifact',  'planeswalker']
def corrected_type(main_type: str) -> str:
    '''
    Преобразует типы карт в укрупненные группы
    '''
    cor_type = 'other'
    for card_type in card_types:
        if card_type in main_type.lower():
            cor_type = card_type
            break
    return cor_type

In [11]:
df['Main type'] = df['Main type'].map(corrected_type)

In [12]:
# Обработка переменных
str_to_int = ['Price', 'Qty', 'Power', 'Strength']
df = df[df['Price'].isna() == False]
df['Price']= df['Price'].apply(lambda x: x if x.find('\n') == -1 else x.split('\n')[1])
df['Price'] = df['Price'].str.replace(' ₽','').astype(int)
df['Qty']= df['Qty'].apply(lambda x: x if x.find('\n') == -1 else x.split('\n')[1])
df['Qty'] = df['Qty'].str.replace(' шт.','').astype(int)
df[['Power', 'Strength']] = df[['Power', 'Strength']].fillna(-1)

In [13]:
# Создание фич на основе дат
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df["Month_cos"] = df["Month"].apply(lambda x: np.cos(x / 12 * 2 * np.pi))

Создание признаков на основе цен карт и их свойств в описании

In [14]:
def get_color_cost(cost:str) -> str:
    '''
    Получение цены и цветовой принадлежности карты
    '''
    try:
        cost = cost.replace('{', '').replace('}', '')
        color = ''.join(re.findall(r'\D+', cost))
        col_len = len(color)
        if col_len == 0:
            color = 'Colorless'
        else:
            color = ''.join([''.join(sorted(set(x))) for x in color])
        noncolor = sum([int(x) for x in re.findall(r'\d+', cost)])
        total_cost = noncolor + len(color)
    except Exception:
        color = 'Other'
        total_cost = 0
    return color, total_cost

In [15]:
def get_description_cost(feature:str) -> str:
    '''
    Получение цены и цветовой принадлежности описания карты
    '''
    color = ''
    total_cost = 0
    costs2 = feature.split('{')[1:]
    # print(costs2)
    try:
        costs = [x.split('}')[0]  for x  in costs2 if len(x)> 0] 
        feature_paid_abilities = len(costs)
        total_cost = 0
        color = []
        for cost in costs:
            if cost.isnumeric():
                total_cost += int(cost)
            else: 
                for c in cost:
                    if not c.isnumeric():
                        total_cost += 1
                        color.append(c)
                    else:
                        total_cost += int(c)
        color = ''.join(sorted(set(color)))
    except Exception:
        pass
    return color, total_cost

In [16]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
ability_cols = df.select_dtypes(include=numerics).columns.tolist()[11:]

In [17]:
df['Colors_features'], df['Features cost'] = zip(*df['Feature'].map(get_description_cost))
df['Colors_dist'], df['Color cost'] = zip(*df['Cost'].map(get_color_cost))
df['Foil'] = df['Foil'].apply(lambda x: 1 if x == 'Фойл' else 0)
df['Total abilities qty'] = df[ability_cols].sum(axis = 1)

D:\Python\Anaconda\lib\site-packages\pandas\core\common.py:228: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


In [18]:
ab_cols = ab_word_cols = action_word_cols = []
for col in tqdm(ability_cols):
    if 'Ability_word_' in col:
        ab_word_cols.append(col)
    elif 'Ability_'  in col :
        ab_cols.append(col)  
    elif 'Action_'  in col :
        action_word_cols.append(col) 
df['Abilities only qty'] = df[ab_cols].sum(axis = 1)
df['Ability words only qty'] = df[ab_word_cols].sum(axis = 1)
df['Action only qty'] = df[action_word_cols].sum(axis = 1)
# df.drop(ab_cols + ab_word_cols + action_word_cols , axis = 1, inplace = True)

In [19]:
# df[['Feature','Colors_features', 'Features cost']].head(10)

In [20]:
# Удаление ненужных и редковстречающихся признаков
cols = [ 'Name1', 'Name2', 'Special1', 'Special2', 'Name_y', 'Feature', 'Cost', 'Stats']
for col in ability_cols:
    if df[col].sum() < 500:
        cols.append(col)
df.drop(cols , axis = 1, inplace = True)

In [21]:
df.rename(columns={'Name_x':'Name'}, inplace = True)
quality_dict = {'NM':4, 'SP':3, 'MP':2, 'HP':1}
df['Quality'] = df['Quality'].map(quality_dict)

In [22]:
df.head()

,Language,Set,Foil,Quality,Rarity,Price,Qty,Name,Type,Illustrator,...,Month,Year,Colors_features,Features cost,Colors_dist,Color cost,Total abilities qty,Abilities only qty,Ability words only qty,Action only qty
0,Английский,Ixalan,0,4,Мифическая,190,5,Admiral Beckett Brass,Legendary Creature — Human Pirate,Jason Rainville,...,9,2017,,0,UBR,4,2226.0,1,1,1
1,Английский,Ixalan,0,3,Мифическая,160,0,Admiral Beckett Brass,Legendary Creature — Human Pirate,Jason Rainville,...,9,2017,,0,UBR,4,2226.0,1,1,1
2,Английский,Ixalan,1,4,Мифическая,1360,0,Admiral Beckett Brass,Legendary Creature — Human Pirate,Jason Rainville,...,9,2017,,0,UBR,4,2226.0,1,1,1
3,Английский,Ixalan,1,2,Мифическая,1100,1,Admiral Beckett Brass,Legendary Creature — Human Pirate,Jason Rainville,...,9,2017,,0,UBR,4,2226.0,1,1,1
4,Английский,Ixalan,0,4,Мифическая,180,5,Admiral Beckett Brass,Legendary Creature — Human Pirate,Jason Rainville,...,9,2017,,0,UBR,4,2226.0,1,1,1


In [23]:
df.to_csv(path + '/merged_df.csv', index = False)